In [1]:
!nvidia-smi

Mon May 13 21:38:44 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     On   | 00000000:01:00.0 Off |                  Off |
| 33%   27C    P8    15W / 260W |      1MiB / 49152MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mne
import pywt
from torch.utils.data import DataLoader, Dataset, Subset
from torch import nn
import torch
import pickle
import os
from datetime import datetime
import sqlite3
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.tensorboard import SummaryWriter
from torchmetrics.functional.classification.accuracy import accuracy

2024-05-13 21:38:46.746839: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-13 21:38:46.800916: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 21:38:47.772726: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Training on device: {device}')

Training on device: cuda


In [4]:
class CWTDataset(Dataset):
    """
    Dataset for EEG data after CWT transformation stored in SQLite database.
    
    Attributes:
        db_path: str - path to SQLite database
        sequence_length: int - length of the sequence
    Methods:
        __len__ - returns the number of samples in the dataset minus the sequence length
        __getitem__ - returns a sample from the dataset
    
    """

    def __init__(self, db_path, sequence_length=4000):
        """
        Constructor for CWTDataset class that initializes the dataset.
        Args:
            db_path: str - path to SQLite database
            sequence_length: int - length of the sequence
        Returns:
            None
        """
        self.db_path = db_path
        self.sequence_length = sequence_length
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        self.cursor.execute("SELECT COUNT(*) FROM wavelet_transforms")
        self.total_samples = self.cursor.fetchone()[0]

    def __len__(self):
        """
        Returns the number of samples in the dataset.
        Args:
            None
        Returns:
            int - number of samples in the dataset minus the sequence length
        """
        return self.total_samples - self.sequence_length + 1

    def __getitem__(self, idx):
        """
        function that returns as many samples as the sequence length
        Args:
            idx: int - index of the sample
        Returns:
            tuple - (cwt_tensor, target_tensor)
        
        """
        query = (
            "SELECT cwt_data, target FROM wavelet_transforms WHERE id BETWEEN ? AND ?"
        )

        self.cursor.execute(query, (idx + 1, idx + self.sequence_length))
        rows = self.cursor.fetchall()

        cwt_sequence = np.stack([pickle.loads(row[0]) for row in rows])

        target = rows[-1][1]

        cwt_tensor = torch.tensor(cwt_sequence, dtype=torch.float32)

        target_tensor = torch.tensor(target, dtype=torch.int64)
        return cwt_tensor, target_tensor

    def __del__(self):
        self.conn.close()

In [5]:
class CWTSubset(Dataset):
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices

    def __getitem__(self, idx):
        row = self.dataset.__getitem__(int(self.indices[idx]))
        return row

    def __len__(self):
        return len(self.indices)

In [6]:
class CWT_EEG(LightningModule):
    def __init__(
            self,
            batch_size,
            sequence_length,
            input_size,
            hidden_size,
            num_layers,
            lr,
            label_smoothing=0,
    ):
        super().__init__()
        self.save_hyperparameters()
        self.hparams.batch_size = batch_size
        self.hparams.input_size = input_size
        self.hparams.sequence_length = sequence_length
        self.hparams.lr = lr
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_of_classes = 3
        self.val_percent = 0.01
        self.loss = torch.nn.CrossEntropyLoss(label_smoothing=label_smoothing)

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, self.num_of_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = hn[-1, :, :]
        out = self.fc(out)

        return out

    # custom
    def count_parameters(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

    # only for HP
    def on_train_start(self):
        self.logger.log_hyperparams(
            self.hparams,
            {
                "hp/train_loss": float("nan"),
                "hp/train_acc": float("nan"),
                "hp/val_loss": float("nan"),
                "hp/val_acc": float("nan"),
            },
        )

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y, task="multiclass", num_classes=self.num_of_classes)

        self.log("hp/train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("hp/train_acc", acc, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y, task="multiclass", num_classes=self.num_of_classes)

        self.log("hp/val_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("hp/val_acc", acc, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def generate_validation_indices(
            self, data_length, num_of_val_samples, sequence_length
    ):
        available_indices = set(range(data_length))
        val_indices = []
        for _ in range(num_of_val_samples):
            if len(available_indices) == 0:
                raise ValueError(
                    "Nie można wygenerować więcej próbek z uwzględnieniem minimalnego dystansu"
                )
            chosen_index = int(np.random.choice(list(available_indices)))
            val_indices.append(chosen_index)
            indices_to_remove = set(
                range(
                    max(0, chosen_index - (2 * sequence_length) - 3),
                    min(data_length, chosen_index + (2 * sequence_length) + 3),
                )
            )
            available_indices.difference_update(indices_to_remove)

        return val_indices

    def generate_train_indices(self, data_length, val_i, sequence_length):
        min_distance = sequence_length + 1
        mask = np.ones(data_length, dtype=bool)
        for index in val_i:
            start = max(0, index - min_distance)
            end = min(data_length, index + min_distance + 1)

            mask[start:end] = False

        training_indices = list(np.where(mask)[0])
        return training_indices

    def setup(self, stage=None):
        self.ds = CWTDataset("df_train_cwt_data.db", self.hparams.sequence_length)
        self.num_val_samples = int(
            len(self.ds) / (4 * self.hparams.sequence_length + 6)
        )

        val_indices = self.generate_validation_indices(
            len(self.ds), self.num_val_samples, self.hparams.sequence_length
        )
        train_indices = self.generate_train_indices(
            len(self.ds), val_indices, self.hparams.sequence_length
        )
        print(
            "percent of val samples",
            len(val_indices) / (len(val_indices) + len(train_indices)),
        )
        self.train_set = CWTSubset(self.ds, train_indices)
        self.val_set = CWTSubset(self.ds, val_indices)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_set,
            batch_size=self.hparams.batch_size,
            num_workers=7,
            shuffle=True,
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.val_set, batch_size=self.hparams.batch_size, num_workers=7
        )

    def get_len_train_val(self):
        self.setup()
        return len(self.train_set), len(self.val_set)

In [7]:
class CWT_EEG_CrossPersonValidation(CWT_EEG):
    def __init__(self, batch_size, sequence_length, input_size, hidden_size, num_layers, lr, label_smoothing=0):
        super().__init__(batch_size, sequence_length, input_size, hidden_size, num_layers, lr, label_smoothing)

    def setup(self, stage=None):
        self.train_set = CWTDataset("./df_train_cwt_data.db", self.hparams.sequence_length)
        self.val_set = CWTDataset("./df_val_cwt_data.db", self.hparams.sequence_length)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_set, batch_size=self.hparams.batch_size, num_workers=14,
                                           shuffle=True)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_set, batch_size=self.hparams.batch_size, num_workers=14)


In [8]:

lr = 0.001

model = CWT_EEG(batch_size=11, sequence_length=10, input_size=640, num_layers=3, hidden_size=3,
                lr=lr).to(device)
logger = TensorBoardLogger("logs", name="CWT_EEG", default_hp_metric=False)
logger.log_hyperparams(model.hparams, {})
trainer = Trainer(
    max_epochs=2,
    logger=logger

)
print(model.get_len_train_val())
trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


OperationalError: no such table: wavelet_transforms

In [9]:
import datetime
from pytorch_lightning import Trainer, loggers

current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
logger = loggers.TensorBoardLogger("logs", name=f"CWT_EEG_{current_time}", default_hp_metric=False)
lr = 0.001
model = CWT_EEG_CrossPersonValidation(batch_size=11, sequence_length=10, input_size=640, num_layers=3, hidden_size=100,
                                   lr=lr).to(device)
logger.log_hyperparams(model.hparams)
trainer = Trainer(
    max_epochs=2,
    logger=logger
)
trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: logs/CWT_EEG_2024-05-13_21-39-18
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type             | Params
------------------------------------------
0 | loss | CrossEntropyLoss | 0     
1 | lstm | LSTM             | 458 K 
2 | fc   | Linear           | 303   
------------------------------------------
458 K     Trainable params
0         Non-trainable params
458 K     Total params
1.835     Total estimated model params size (MB)


Epoch 0:   1%|        | 91/10818 [00:00<01:19, 135.54it/s, v_num=0, hp/train_loss_step=1.170, hp/train_acc_step=0.273]

/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
model.hparams

In [22]:
from ray import tune
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

def train_model(config):
    model = CWT_EEG_CrossPersonValidation(
        batch_size=config["batch_size"],  # hiperparametr rozmiaru batcha
        sequence_length=config["sequence_length"],  # hiperparametr długości sekwencji
        input_size=640,  # hiperparametr rozmiaru wejścia
        hidden_size=config["hidden_size"],  # hiperparametr rozmiaru stanu ukrytego
        num_layers=config["num_layers"],  # hiperparametr liczby warstw
        lr=config["lr"],  # hiperparametr prędkości uczenia
        label_smoothing=0  # hiperparametr wygładzania etykiet
    )

    # Callbacks
    checkpoint_callback = ModelCheckpoint(
        monitor='hp/val_loss',
        dirpath='model_checkpoints',
        filename='model-{epoch:02d}-{hp/val_loss:.2f}',
        save_top_k=3,
        mode='min'
    )
    early_stop_callback = EarlyStopping(
        monitor='hp/val_loss',
        min_delta=0.00,
        patience=3,
        verbose=False,
        mode='min'
    )

    trainer = Trainer(
        max_epochs=10,
        callbacks=[checkpoint_callback, early_stop_callback],
    )

    trainer.fit(model)

search_space = {
    "batch_size": tune.choice([8, 16, 32, 64]),
    "sequence_length": tune.choice([10, 100, 200]),
    "hidden_size": tune.choice([256, 512, 1024]),
    "num_layers": tune.choice([1, 2, 3]),
    "lr": tune.loguniform(1e-4, 1e-1),
}

analysis = tune.run(
    train_model,
    config=search_space,
    num_samples=10,
    resources_per_trial={"cpu": 1, "gpu": 1}
)


2024-05-13 22:15:49,927	INFO tune.py:614 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


2024-05-13 22:15:55,449	ERROR tune_controller.py:1331 -- Trial task failed for trial train_model_974c0_00000
Traceback (most recent call last):
  File "/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/ray/_private/worker.py", line 2623, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, tim

2024-05-13 22:16:02,054	ERROR tune_controller.py:1331 -- Trial task failed for trial train_model_974c0_00001
Traceback (most recent call last):
  File "/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/ray/_private/worker.py", line 2623, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, tim

In [10]:
import os
from ray import tune
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from pytorch_lightning import Trainer, LightningModule
import torch


def train_cwt_eeg(config):
    model = CWT_EEG_CrossPersonValidation(config)
    trainer = Trainer(
        max_epochs=10,
        progress_bar_refresh_rate=0,  # Disable progress bar for clearer logs
        callbacks=[TuneReportCallback({"loss": "ptl/val_loss"}, on="validation_end")],
    )
    trainer.fit(model)


search_space = {
    "batch_size": tune.choice([8, 16, 32, 64]),
    "sequence_length": tune.choice([10, 100, 200]),
    "input_size": 640,  # This is fixed for our dataset
    "hidden_size": tune.choice([256, 512, 1024]),
    "num_layers": tune.choice([1, 2, 3]),
    "lr": tune.loguniform(1e-5, 1e-1),
    "label_smoothing": 0,
}
# Uruchomienie procesu optymalizacji
analysis = tune.run(
    train_cwt_eeg,
    config=search_space,
    num_samples=10,
    resources_per_trial={"cpu": 1, "gpu": 1},  # Adjust based on your system's resources
)

print("Best hyperparameters found were: ", analysis.best_config)

/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-13 21:39:24,055	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-13 21:39:24,126	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-13 21:39:27,385	INFO worker.py:1749 -- Started a local Ray instance.
2024-05-13 21:39:28,115	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2024-05-13 21:39:28,118	INFO tune.py:614 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks a

2024-05-13 21:39:31,237	ERROR tune_controller.py:1331 -- Trial task failed for trial train_cwt_eeg_82d7b_00000
Traceback (most recent call last):
  File "/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/ray/_private/worker.py", line 2623, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, t

2024-05-13 21:39:34,378	ERROR tune_controller.py:1331 -- Trial task failed for trial train_cwt_eeg_82d7b_00001
Traceback (most recent call last):
  File "/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/danielkleczykkleczynski/miniconda3/envs/EEG311M/lib/python3.11/site-packages/ray/_private/worker.py", line 2623, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, t

ValueError: To fetch the `best_config`, pass a `metric` and `mode` parameter to `tune.run()`. Alternatively, use the `get_best_config(metric, mode)` method to set the metric and mode explicitly.

(train_model pid=53252) GPU available: True (cuda), used: True
(train_model pid=53252) TPU available: False, using: 0 TPU cores
(train_model pid=53252) IPU available: False, using: 0 IPUs
(train_model pid=53252) HPU available: False, using: 0 HPUs
(train_model pid=53252) Missing logger folder: /tmp/ray/session_2024-05-13_21-39-24_464251_49542/artifacts/2024-05-13_22-10-12/train_model_2024-05-13_22-10-12/working_dirs/train_model_ce18c_00000_0_batch_size=32,hidden_size=256,lr=0.0016,num_layers=2,sequence_length=200_2024-05-13_22-10-12/lightning_logs
(train_model pid=53252) 2024-05-13 22:10:16.704882: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(train_model pid=53252) 2024-05-13 22:10:16.762331: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary 

In [ ]:
# Utworzenie folderu dla logów
writer = SummaryWriter('./lightning_logs/CWT_EEG')

In [ ]:
dummy_input = torch.randn(11, 10, 640).to(device)  # Przykładowe dane wejściowe (batch_size, input_size)
writer.add_graph(model, dummy_input)
writer.close()

In [ ]:
import torch
import torch.nn as nn


class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SimpleLSTM, self).__init__()
        self.hidden_size = hidden_size

        # Warstwa LSTM
        self.lstm = nn.LSTM(input_size, hidden_size)

    def forward(self, input_seq):
        # Inicjalizacja stanu ukrytego i stanu komórki
        # (num_layers * num_directions, batch, hidden_size)
        h0 = torch.zeros(1, input_seq.size(1), self.hidden_size)
        c0 = torch.zeros(1, input_seq.size(1), self.hidden_size)

        # Przejście przez LSTM
        lstm_out, _ = self.lstm(input_seq, (h0, c0))

        return lstm_out


# Parametry modelu
input_size = 20  # Wymiary wejściowe (np. cechy)
hidden_size = 10  # Wymiary stanu ukrytego LSTM

# Tworzenie instancji modelu
model = SimpleLSTM(input_size, hidden_size)

dummy_input = torch.randn(1, 10, 20)  # Przykładowe dane wejściowe (batch_size, input_size)
writer.add_graph(model, dummy_input)
writer.close()
# Wyświetlenie struktury modelu
print(sum(p.numel() for p in model.parameters() if p.requires_grad))


In [ ]:
def create_train_val_split(data_length, val_percent, sequence_length):
    available_lenght = data_length - sequence_length
    val_count = int(val_percent * available_lenght)
    all_indices = np.arange(available_lenght)
    val_indices = np.sort(np.random.choice(all_indices, size=val_count, replace=False))
    mask = np.ones(available_lenght, dtype=bool)
    for idx in val_indices:
        start = max(0, idx - sequence_length + 1)
        end = min(available_lenght, idx + sequence_length)
        mask[start:end] = False
    train_indices = np.where(mask)[0]
    # convert evry number to int
    train_indices = list(train_indices)
    val_indices = list(val_indices)

    return train_indices, val_indices

In [ ]:
from tensorboard.backend.event_processing import event_accumulator
import matplotlib.pyplot as plt


def load_tensorboard_scalars(logdir):
    # Tworzy akumulator do odczytu danych
    ea = event_accumulator.EventAccumulator(logdir,
                                            size_guidance={event_accumulator.SCALARS: 0})  # 0 = bez ograniczeń
    ea.Reload()  # Wczytaj wszystkie dane z dysku

    # Odczyt danych skalarnych
    scalars = {}
    for tag in ea.Tags()['scalars']:
        events = ea.Scalars(tag)
        scalars[tag] = [(e.wall_time, e.step, e.value) for e in events]

    return scalars


# Ścieżka do katalogu z logami TensorBoard
logdir = '/home/daniel/repos/Decoding_of_EEG/lightning_logs/CWT_EEG/version_51/events.out.tfevents.1713204451.pop-os.32833.5'

# Wczytanie danych
scalars = load_tensorboard_scalars(logdir)

# Przykład wyświetlenia danych skalarnych
tag = 'hp/train_acc_step'  # Zmień na odpowiedni tag, który chcesz wyświetlić
times, steps, values = zip(*scalars[tag])

plt.figure(figsize=(10, 5))
plt.plot(steps, values, label=tag)
plt.xlabel('Krok')
plt.ylabel('Wartość')
plt.title('Wykres danych z TensorBoard')
plt.legend()
plt.show()


In [ ]:
tag = 'hp/val_acc_step'  # Zmień na odpowiedni tag, który chcesz wyświetlić
times, steps, values = zip(*scalars[tag])

plt.figure(figsize=(10, 5))
plt.plot(steps, values, label=tag)
plt.xlabel('Krok')
plt.ylabel('Wartość')
plt.title('Wykres danych z TensorBoard')
plt.legend()
plt.show()

In [ ]:
import numpy as np


def generate_validation_indices(data_length, num_of_val_samples, sequence_length):
    # Ustal minimalną odległość pomiędzy indeksami
    min_distance = sequence_length + 1
    # Utwórz set możliwych indeksów
    available_indices = set(range(data_length))

    chosen_indices = []
    for i in range(num_of_val_samples):
        if not available_indices:
            raise ValueError("Nie można wygenerować więcej próbek z uwzględnieniem minimalnego dystansu")

        # Losuj indeks z dostępnych indeksów
        chosen_index = np.random.choice(list(available_indices))
        chosen_indices.append(chosen_index)

        # Oblicz zakres indeksów do usunięcia
        start = max(0, chosen_index - (2 * min_distance))
        end = min(data_length, chosen_index + min_distance)

        # Usuń indeksy zbyt blisko wybranego indeksu z zbioru available_indices
        for idx in range(start, end + 1):
            available_indices.discard(idx)  # discard nie zgłasza błędu, jeśli element nie istnieje

        # Wydrukuj postęp
        print(
            f"Postęp: {i + 1}/{num_of_val_samples} indeksów wygenerowanych. ({(i + 1) / num_of_val_samples * 100:.2f}%)")

    return chosen_indices


# Przykładowe wywołanie funkcji
data_length = 10000
num_of_val_samples = 10
sequence_length = 200

try:
    val_indices = generate_validation_indices(data_length, num_of_val_samples, sequence_length)
    print("Wygenerowane indeksy walidacyjne:", val_indices)
    print("len(val_indices):", len(val_indices))
except ValueError as e:
    print(e)


In [ ]:
import numpy as np


def generate_validation_indices(data_length, num_of_val_samples, sequence_length):
    # Ustal minimalną odległość pomiędzy indeksami
    min_distance_forward = sequence_length + 1
    min_distance_backward = 2 * sequence_length + 1
    print("min_distance_forward:", min_distance_forward)
    print("min_distance_backward:", min_distance_backward)
    # Utwórz listę możliwych indeksów
    available_indices = list(range(data_length))

    chosen_indices = []
    for i in range(num_of_val_samples):
        if not available_indices:
            raise ValueError("Nie można wygenerować więcej próbek z uwzględnieniem minimalnego dystansu")
        array = np.array(available_indices)
        # Losuj indeks z dostępnych indeksów
        chosen_index = np.random.choice(available_indices)
        chosen_indices.append(chosen_index)

        # Oblicz zakres indeksów do usunięcia
        start = max(0, chosen_index - min_distance_backward)
        end = min(data_length, chosen_index + min_distance_forward)

        # Usuń indeksy zbyt blisko wybranego indeksu
        available_indices = [idx for idx in available_indices if idx < start or idx > end]
        print("len(available_indices):", len(available_indices))
        # Wydrukuj postęp
        print(
            f"Postęp: {i + 1}/{num_of_val_samples} indeksów wygenerowanych. ({(i + 1) / num_of_val_samples * 100:.2f}%)")

    return chosen_indices


# Przykładowe wywołanie funkcji
data_length = 100
num_of_val_samples = 100
sequence_length = 200

try:
    val_indices = generate_validation_indices(data_length, num_of_val_samples, sequence_length)
    print("Wygenerowane indeksy walidacyjne:", val_indices)
    print("len(val_indices):", len(val_indices))
except ValueError as e:
    print(e)


In [ ]:
import numpy as np


def generate_validation_indices(data_length, num_of_val_samples, sequence_length):
    min_distance = sequence_length + 1  # Minimalna odległość pomiędzy indeksami
    available_indices = set(range(data_length))  # Tworzymy zbiór dostępnych indeksów

    chosen_indices = []
    for _ in range(num_of_val_samples):
        if len(available_indices) == 0:
            raise ValueError("Nie można wygenerować więcej próbek z uwzględnieniem minimalnego dystansu")

        chosen_index = np.random.choice(list(available_indices))  # Losujemy z dostępnych indeksów
        chosen_indices.append(chosen_index)

        # Usuwamy indeksy w zakresie `sequence_length` w obie strony od wybranego indeksu
        indices_to_remove = set(range(max(0, chosen_index - (2 * sequence_length) - 3),
                                      min(data_length, chosen_index + (2 * sequence_length) + 3)))
        available_indices.difference_update(indices_to_remove)  # Aktualizujemy zbiór dostępnych indeksów

    return chosen_indices


# Przykładowe wywołanie funkcji
data_length = 1000
num_of_val_samples = 10
sequence_length = 50

try:
    val_indices = generate_validation_indices(data_length, num_of_val_samples, sequence_length)
    print("Wygenerowane indeksy walidacyjne:", val_indices)
except ValueError as e:
    print(e)


In [ ]:
import numpy as np


def generate_mask(data_length, chosen_indices, sequence_length):
    # Ustal minimalną odległość pomiędzy indeksami
    min_distance = sequence_length + 1
    # Utwórz maskę początkową ze wszystkimi wartościami ustawionymi na True
    mask = np.ones(data_length, dtype=bool)

    # Iteruj przez każdy wybrany indeks walidacyjny
    for index in chosen_indices:
        # Ustal zakres indeksów, które należy ustawić na False
        start = max(0, index - min_distance)
        end = min(data_length, index + min_distance)

        # Ustaw odpowiednie wartości w masce na False
        mask[start:end] = False

    # Zwróć indeksy, gdzie maska jest True, czyli indeksy zbioru treningowego
    training_indices = np.where(mask)[0]  # np.where(mask) zwraca tuple, [0] wyciąga array z indeksami
    return training_indices



In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def plot_val_indices(data_length, val_indices, sequence_length):
    # Ustal minimalną odległość pomiędzy indeksami
    min_distance = sequence_length + 1

    # Inicjalizacja figury
    plt.figure(figsize=(10, 2))
    plt.title("Rozkład indeksów walidacyjnych i ich zakresy")
    plt.xlabel("Indeksy danych")
    plt.ylabel("Wartość (dla wizualizacji)")

    # Rysowanie linii dla całej długości danych
    plt.plot([0, data_length - 1], [1, 1], label='Dane', color='blue')

    # Rysowanie punktów dla walidacyjnych indeksów
    for index in val_indices:
        plt.scatter([index], [1], color='red')  # punkt walidacyjny
        start = max(0, index - min_distance)
        end = min(data_length, index + min_distance)
        plt.axvspan(start, end, color='red', alpha=0.3)  # zakres wokół punktu)

    plt.legend(['Dane', 'Indeksy walidacyjne i zakres'])
    plt.grid(True)
    plt.show()


# Przykładowe wywołanie funkcji
data_length = 200
num_of_val_samples = 10
sequence_length = 4

try:
    val_indices = generate_validation_indices(data_length, num_of_val_samples, sequence_length)
    plot_val_indices(data_length, val_indices, sequence_length)
except ValueError as e:
    print(e)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def plot_train_val_indices(data_length, train_indices, val_indices, sequence_length):
    # Ustal minimalną odległość pomiędzy indeksami
    min_distance = sequence_length + 1

    # Inicjalizacja figury
    plt.figure(figsize=(10, 2))
    plt.title("Rozkład indeksów walidacyjnych i ich zakresy")
    plt.xlabel("Indeksy danych")
    plt.ylabel("Wartość (dla wizualizacji)")

    # Rysowanie linii dla całej długości danych
    plt.plot([0, data_length - 1], [1, 1], label='Dane', color='blue')

    # Rysowanie punktów dla walidacyjnych indeksów
    for index in val_indices:
        plt.scatter([index], [1], color='red')  # punkt walidacyjny
        start = max(0, index - min_distance)
        end = min(data_length, index + min_distance)
        plt.axvspan(start, end, color='red', alpha=0.3)  # zakres wokół punktu)
    # if train_indices is not empty:
    for index in train_indices:
        plt.scatter([index], [1], color='red')  # punkt walidacyjny
        start = index
        end = min(data_length, index + min_distance)
        plt.axvspan(start, end, color='blue', alpha=0.1)  # zakres wokół punktu)

    plt.legend(['Dane', 'Indeksy walidacyjne i zakres'])
    plt.grid(True)
    plt.show()


# Przykładowe wywołanie funkcji
data_length = 200
num_of_val_samples = 10
sequence_length = 5

try:
    val_indices = generate_validation_indices(data_length, num_of_val_samples, sequence_length)
    train_indices = generate_mask(data_length, val_indices, sequence_length)
    plot_val_indices(data_length, val_indices, sequence_length)
    plot_train_val_indices(data_length, train_indices, val_indices, sequence_length)
except ValueError as e:
    print(e)

In [ ]:
import numpy as np


def generate_mask(data_length, chosen_indices, sequence_length):
    # Ustal minimalną odległość pomiędzy indeksami
    min_distance = sequence_length + 1
    # Utwórz maskę początkową ze wszystkimi wartościami ustawionymi na True
    mask = np.ones(data_length, dtype=bool)

    # Iteruj przez każdy wybrany indeks walidacyjny
    for index in chosen_indices:
        # Ustal zakres indeksów, które należy ustawić na False
        start = max(0, index - min_distance)
        end = min(data_length, index + min_distance)

        # Ustaw odpowiednie wartości w masce na False
        mask[start:end] = False

    # Zwróć indeksy, gdzie maska jest True, czyli indeksy zbioru treningowego
    training_indices = np.where(mask)[0]  # np.where(mask) zwraca tuple, [0] wyciąga array z indeksami
    return training_indices


# Przykładowe wywołanie funkcji
data_length = 200
num_of_val_samples = 10
sequence_length = 4

try:
    val_indices = generate_validation_indices(data_length, num_of_val_samples, sequence_length)
    training_indices = generate_mask(data_length, val_indices, sequence_length)
    plot_validation_indices(data_length, training_indices, val_indices, sequence_length)
    print("Wygenerowane indeksy walidacyjne:", val_indices)
    print("Indeksy zbioru treningowego:", training_indices)
except ValueError as e:
    print(e)


In [ ]:
randomlist = [1, 2, 9, 10]
index = np.random.choice(randomlist)
index

In [ ]:
# Przykładowe wywołanie funkcji
data_length = 1000
num_of_val_samples = 10
sequence_length = 30

try:
    val_indices = generate_validation_indices(data_length, num_of_val_samples, sequence_length)
    training_indices = generate_mask(data_length, val_indices, sequence_length)
    print("len(training_indices):", len(training_indices))
except ValueError as e:
    print(e)


In [ ]:
!pip install sidekit
